In [1]:
import torch
import pytorch_lightning as pl
import transformers
import torchmetrics

import pandas as pd
import os
import json

from model import T5MultiTask
from data_module import TolokaDataModule

/home/stc/rybin-as/miniconda3/envs/asr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# proxy
os.environ["http_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["https_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["ftp_proxy"] = "http://proxy.ad.speechpro.com:3128"

In [3]:
t5 = transformers.T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small-chitchat2")
tokenizer = transformers.AutoTokenizer.from_pretrained("cointegrated/rut5-small-chitchat2", truncation_side='left', padding_side='right')

with open('/home/stc/persona/data/TlkPersonaChatRus/spec_tokens.json') as spec_tokens_config:
    spec_tokens = json.load(spec_tokens_config)
tokenizer.add_special_tokens(
            {"additional_special_tokens": [spec_tokens[k] for k in spec_tokens]}
        )

In [ ]:
datamodule=TolokaDataModule(
    data_dir='/home/stc/persona/data/TlkPersonaChatRus',
    datasets=['next_gk'], #'next_answer', 'current_gk', 'next_gk'
    tokenizer_d=tokenizer,
    spec_tokens=spec_tokens,
    train_batch_size=64,
    val_batch_size=64,
    test_batch_size=64,
)

/home/stc/rybin-as/miniconda3/envs/asr/lib/python3.8/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [ ]:
model = T5MultiTask(
    model=t5,
    tokenizer=tokenizer,
    datasets=['next_gk'],
    lr=5e-5,
    num_warmup_steps=100,
    pooling="cls",
    distance="cosine",
    scale=20,
    val_batch_size=64,
)

/home/stc/rybin-as/miniconda3/envs/asr/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(


In [ ]:
# logger
logger = pl.loggers.comet.CometLogger(
    api_key='sEJsZrYjwc0gxxUAUGQNBwTsb',
    save_dir='/home/stc/persona/logs/t5',
    project_name='T5',
    experiment_name='test',
)

CometLogger will be initialized in online mode


In [ ]:
# trainer
trainer = pl.Trainer(
    max_epochs=30,
    accelerator="gpu",
    devices=1,
    gradient_clip_val=1,
    logger=logger,
    num_sanity_val_steps=10,
)
trainer.fit(model, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type                       | Params
------------------------------------------------------------------
0 | transformer        | T5ForConditionalGeneration | 64.8 M
1 | cross_entropy_loss | CrossEntropyLoss           | 0     
------------------------------------------------------------------
64.8 M    Trainable params
0         Non-trainable params
64.8 M    Total params
259.053   Total estimated model params size (MB)
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure al

/home/stc/rybin-as/miniconda3/envs/asr/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:366: UserWarning: One of given dataloaders is None and it will be skipped.
  rank_zero_warn("One of given dataloaders is None and it will be skipped.")
Loading cached shuffled indices for dataset at /home/stc/persona/data/TlkPersonaChatRus/next_gk/train/cache-018e6ebcccf5db38.arrow


  7%|▋         | 172/2524 [00:02<00:33, 69.41ba/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  9%|▉         | 228/2524 [00:03<00:34, 66.88ba/s]

...

100%|██████████| 2524/2524 [00:32<00:00, 76.68ba/s]


Epoch 0: 100%|██████████| 2524/2524 [06:08<00:00,  6.84it/s, loss=4.05, v_num=77c9]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 2524/2524 [06:09<00:00,  6.83it/s, loss=4.05, v_num=77c9]


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/anpopaicoconat/t5/77081e1caf54435db8e08141f4b277c9
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_loss [50]         : (3.9463043212890625, 5.183550834655762)
COMET INFO:     val_mrr@64_next_gk [50] : (0.06100258231163025, 0.17642925679683685)
COMET INFO:     val_r1@64_next_gk [50]  : (0.0, 0.09375)
COMET INFO:     val_r5@64_next_gk [50]  : (0.0625, 0.203125)
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:     Name         : test
COMET INFO:   Parameters:
COMET INFO:     data_dir                   : /home/stc/persona/data/TlkPersonaChatRus
COMET INFO:     datasets                   : ['next_gk']
COMET INFO:     distance                   : cosine
COMET INFO:     lr                         : 5e-05

In [ ]:
torch.tensor([1]).to(torch.float32)

tensor([1.])